In [1]:
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler

n = 16
q = 32768
t = 256
sigma = 8
delta = q//t

S.<x> = PolynomialRing(ZZ)
R.<zeta> = S.quotient(S.ideal(x**(n) + 1, q))

SampleSmallPoly = lambda :  R(DiscreteGaussianDistributionPolynomialSampler(ZZ['x'], n, sigma)())

def round_to_nearest(x, delta):
    # a // b = floor(a/b), where RHS is rational division
    # (a + (b//2))//b ~ floor(a/b + 1/2) = round_to_nearest_int(a/b)
    return (x + delta//2) // delta

def poly_round(poly, delta):
    return R(lift(poly).map_coefficients(lambda c: round_to_nearest(c, delta)))


In [2]:
def keygen():
    return SampleSmallPoly()

def enc(m, s):
    a = R.random_element()
    e = SampleSmallPoly()
    return (a, a*s + e + delta*m)

def dec(ct, s):
    (a, b) = ct
    partial_dec = b - a * s
    return R(partial_dec.lift().map_coefficients(lambda c: round_to_nearest(c, delta)))

def hom_sum(ct0, ct1):
    (a0, b0) = ct0
    (a1, b1) = ct1
    return (a0+a1, b0+b1)

def hom_prod_and_dec(ct0, ct1, s):
    (u0, v0) = ct0
    (u1, v1) = ct1
    num = v0*v1 - s * (u0*v1+v0*u1) + s*s*u0*u1
    return poly_round(num, delta**2)
    

In [8]:
s = keygen()
m0 = 3
m1 = 2
ct0 = enc(m0, s)
ct1 = enc(m1, s)
hom_prod_and_dec(ct0, ct1, s)

6